<a href="https://colab.research.google.com/github/wolfram-roemhild/dcgan/blob/master/start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/wolfram-roemhild/dcgan start
!ls

Cloning into 'start'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 260 (delta 14), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (260/260), 1.86 MiB | 3.37 MiB/s, done.
Resolving deltas: 100% (88/88), done.
sample_data  start


In [2]:
!cp -r start/* .
!ls

download.py  main.py   README.md    start	 utils.py
img	     model.py  sample_data  start.ipynb


In [3]:
!pip3 install google-colab

In [4]:
!pip3 install cufflinks

    100% |████████████████████████████████| 38.4MB 1.0MB/s 
  Running setup.py bdist_wheel for retrying ... - done
  Stored in directory: /root/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
  Found existing installation: plotly 1.12.12
    Uninstalling plotly-1.12.12:
      Successfully uninstalled plotly-1.12.12


In [5]:
!pip3 install pymc3

    100% |████████████████████████████████| 276kB 19.3MB/s 
  Found existing installation: joblib 0.13.0
    Uninstalling joblib-0.13.0:
      Successfully uninstalled joblib-0.13.0


In [6]:
!pip3 install featuretools

    100% |████████████████████████████████| 8.9MB 4.1MB/s 
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0


In [7]:
!pip3 install spacy

    100% |████████████████████████████████| 13.9MB 2.9MB/s 
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


In [10]:
!pip3 install yellowbrick

    100% |████████████████████████████████| 12.6MB 3.5MB/s 
  Found existing installation: matplotlib 3.0.2
    Uninstalling matplotlib-3.0.2:
      Successfully uninstalled matplotlib-3.0.2


In [11]:
!pip3 install tensorlayer

In [9]:
!python download.py celebA

download.py  ['celebA']
 download_celeb_a ./data
save_path ./data/img_align_celeba.zip
./data/img_align_celeba.zip: 44.1kB [00:14, 3.12kB/s]
dirpath ./data


In [8]:
!ls data

celebA


In [11]:
!pip3 install scikit-image

    100% |████████████████████████████████| 2.0MB 9.1MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
!python main.py

"epoch": 25
"learning_rate": 0.0002
"beta1": 0.5
"train_size": inf
"batch_size": 64
"image_size": 108
"output_size": 64
"sample_size": 64
"c_dim": 3
"sample_step": 500
"save_step": 500
"dataset": celebA
"checkpoint_dir": checkpoint
"sample_dir": samples
"is_train": False
"is_crop": True
"visualize": False
"h": False
"help": False
"helpfull": False
"helpshort": False
--------------------
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1334, in _do_call
    return fn(*args)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1319, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Cannot assign a device for operation generator/g/h0/lin/W/Initializer/truncat